<a href="https://colab.research.google.com/github/MorganPeju/inf8225_project/blob/main/Projet_INF8225.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projet INF8225

### Equipe :
- Renaud Lespérance
- Gabriel Poupart
- Chu Ka Hei
- Morgan Péju

In [2]:
1

1